In [52]:
import pandas as pd
import os
import json
import re
import numpy as np
from bs4 import BeautifulSoup

## Below Section Small modules are Created for each features that is Scrapped From Html
### Exception Handling is done in All Modules to Insert Record Smoothly in DataFrame without any Error

### 1: House Address ,City ,ZipCode Information

In [53]:
def get_house_details(content,realestate_df,index):
    
    try:
        #Street Address
        street_Addr = soup.find(class_='street-address').get_text()
        realestate_df.loc[index]['Address'] = street_Addr

        #City
        city = soup.find(class_='locality').get_text()[:-2]
        realestate_df.loc[index]['City'] = city 

        #State
        state = soup.find(class_='region').get_text()
        realestate_df.loc[index]['State'] = state

        #Zipcode
        zipcode = soup.find(class_='postal-code').get_text()
        realestate_df.loc[index]['ZipCode'] = zipcode
    except Exception as e:
        pass

### 2: Get Detail House Information

In [54]:
#get bed and bath information
def bedsDetail(content):
    noOfBeds = soup.findAll("div", attrs = {'class': 'info-block','data-rf-test-id' : 'abp-beds'})
    noOfBedsList = []
    for value in noOfBeds:
        noOfBedsList.append(value.get_text())
    
    noOfBedsList2 = []
    for value2 in noOfBedsList:
        noOfBedsList2.append(value2[0:1])
    return noOfBedsList2

def bathsDetail(content):
    noOfBaths = soup.findAll("div", attrs = {'class': 'info-block','data-rf-test-id' : 'abp-baths'})
    noOfBathsList = []
    for value in noOfBaths:
        noOfBathsList.append(value.get_text())
    
    noOfBathsList2 = []
    for value2 in noOfBathsList:
        noOfBathsList2.append(value2[0:1])
    return noOfBathsList2


def parkingDetail(content):
    info = content.findAll('span',attrs = {'class':'entryItemContent'})
    parkingDict = {}
    for tag in info:                        
        if re.search(r'Parking Spaces:.*',tag.get_text()):
            a = tag.get_text().split(':')
            for value in a:
                parkingDict[a[0]] = a[1]
    return parkingDict 

def garageDetail(content):
    info = content.findAll('span',attrs = {'class':'entryItemContent'})
    garageDict = {}
    for tag in info:                        
        if re.search(r'Garage Spaces:.*',tag.get_text()):
            b = tag.get_text().split(':')
            for value in b:
                garageDict[b[0]] = b[1]
    return garageDict

In [55]:
#Get'Cooling' Detail
def Cooling_Detail(content):
    info = content.findAll('span',attrs = {'class':'entryItemContent'})
    Cooling_dict = {}
    for tag in info: 
        if re.search(r'Cooling.*',tag.get_text()):
            a = tag.get_text().split(':')
            for value in a:
                Cooling_dict[a[0]] = a[1]
    return Cooling_dict

#Get'Heating' Detail                
def Heating_Detail(content):
    info = content.findAll('span',attrs = {'class':'entryItemContent'})
    Heating_dict = {}
    for tag in info:                        
        if re.search(r'Heating.*',tag.get_text()):
            b = tag.get_text().split(':')
            for value in b:
                Heating_dict[b[0]] = b[1]
    return Heating_dict

def LivingSqFt_InfoBlock(content):
    divs = content.findAll('div',attrs = {'class':'info-block sqft'})
    for div in divs:
        for span in div.findAll('span',attrs = {'class':'statsValue'}):
            return span.get_text().replace(",","")
        
#Get'Living Sq. Ft.' Detail                
def LivingSqFt_Detail(content):
    info = content.findAll('span',attrs = {'class':'entryItemContent'})
    LivingSqFt_dict = {}
    for tag in info:                        
        if re.search(r'Living Sq. Ft.: .*',tag.get_text()):
            c = tag.get_text().split(':')
            for value in c:
                LivingSqFt_dict[c[0]] = c[1]

    if 'Living Sq. Ft.' not in LivingSqFt_dict:
        LivingSqFt_dict['Living Sq. Ft.'] = LivingSqFt_InfoBlock(content)
        
    return LivingSqFt_dict 


#Get 'Lot Acres' Detail
def LotAcres_Detail(content):
    info = content.findAll('span',attrs = {'class':'entryItemContent'})
    LotAcres_dict = {}
    for tag in info:                        
        if re.search(r'Lot Acres: .*',tag.get_text()):
            d = tag.get_text().split(':')
            for value in d:
                LotAcres_dict[d[0]] = d[1]
    return LotAcres_dict 


In [56]:
# Property Type
def PropertyDetail(content):
    PropertyType = content.findAll('div',attrs={'class':'facts-table'})
    PropertyType=PropertyType[0].contents[7].contents[1].get_text()
    dct_PropertyDetail = {}
    dct_PropertyDetail['PropertyType']=PropertyType
    return dct_PropertyDetail 

### 3: get School details Information

In [57]:
def schoolDetail(content):
    results = content.find(id='content').findAll("div", {"class": "schools-content"})
    schoolnames =results[0].findAll("div", {"class": "school-title"})
    schoolratings =results[0].findAll("div", {"class": "gs-rating-text"})    
    namelist =[]
    rating_val =[]
    for x in schoolnames :
        namelist.append(x.text)
    for rating in schoolratings:
        rating_val.append(rating.findAll("span")[0].text)
    school_info =zip(namelist,rating_val)
    dct_schoolDetail =dict(school_info)
    return dct_schoolDetail

### 4:  Walk Score, Transit Score And Bike Score

In [58]:
# get Walk score, transit score, Bike score
def transitDetail(content):
    scores = content.findAll('div',attrs={'class':'percentage','data-rf-test-name':'ws-percentage'})
    walk_score = scores[0].contents[0].get_text()
    transit_score = scores[1].contents[0].get_text()
    bike_score = scores[2].contents[0].get_text()
    dct_TransitDetail = {}
    dct_TransitDetail['WalkScore']=walk_score
    dct_TransitDetail['TransitScore']=transit_score
    dct_TransitDetail['BikeScore']=bike_score
    return dct_TransitDetail


### 5: HOA Information

In [59]:
# HOA Details
def HOADetail(content):
    HOA = content.findAll('span',attrs={'class':'content text-right'})
    HOA = HOA[0].contents[0]
    dct_HOADetail = {}
    if HOA[0]=='$':
        dct_HOADetail['Hoa/Month']=HOA
    else:
        dct_HOADetail['Hoa/Month']='NULL'
    return dct_HOADetail

### 6: Built Year and Renovate Year

In [60]:
# Built Year and Renovated Year
def BuiltDetail(content):
    Built = content.findAll('div',attrs={'class':'facts-table'})
    BuiltYear=Built[0].contents[8].contents[1].get_text()
    RenovatedYear=Built[0].contents[9].contents[1].get_text()
    dct_BuiltDetail = {}
    dct_BuiltDetail['YearBuilt']=BuiltYear
    dct_BuiltDetail['YearRenovated']=RenovatedYear
    return dct_BuiltDetail   

### 7:  RedFin Estimate ,Estimate ForSold Houses ,Selling  Price ,House Status, MLSListing, Description

In [61]:
#get Redfin Estimate
def redfinEstimateValue(content):
    estimateFinder = content.findAll('span' , attrs = {'data-rf-test-id' : 'avmLdpPrice'})
    redfinEstimateList = []
    for values in estimateFinder:
        redfinEstimateList.append(values.get_text())
        
    redfinEstimate = []
    for i in redfinEstimateList:
        redfinEstimate.append(i.split(':')[1])
        
    
        
    return redfinEstimate

In [62]:
#redfin estimate for sold houses
def redfinEstimateForSoldHouses(content):
    redfinEstimaterSoldHouses = soup.findAll('div', attrs = {'class': 'info-block avm', 'data-rf-test-id': 'avm-price'})
    
    redfinEstimateList = []
    for value in redfinEstimaterSoldHouses:
        redfinEstimateList.append(value.get_text())
        
    
    
    redfinEstimateValue =redfinEstimateList[0].replace("Redfin Estimate", "")
    redfinEstimateValue =redfinEstimateValue.replace(",", "")
    redfinEstimateValue =redfinEstimateValue.replace("$", "")

    redfinEstimateSHList = []
    redfinEstimateSHList.append(redfinEstimateValue)

    return redfinEstimateSHList

In [63]:
#get Selling Price
def sellingPrice(content):
    sellingPrice = soup.findAll("div", attrs = {"class": "statsValue"})

    sellingPriceList = []
    sellingPriceList.append(sellingPrice[0].get_text())
    
    return sellingPriceList

In [64]:
#sold houses selling price
def sellingPriceForSoldHouses(content):
    sellingPriceScraper = soup.findAll('div', attrs = {'class': 'info-block price', 'data-rf-test-id': 'abp-price'})

    sellingPriceSH = []
    for value in sellingPriceScraper:
        sellingPriceSH.append(value.get_text())
    
    sellingPriceSoldHouses = sellingPriceSH[0].replace("Sold Price", "")
    sellingPriceSoldHouses = sellingPriceSoldHouses.replace(",", "")
    sellingPriceSoldHouses = sellingPriceSoldHouses.replace("$", "")

    sellingPriceSoldHouseList = []
    sellingPriceSoldHouseList.append(sellingPriceSoldHouses)

    return sellingPriceSoldHouseList

In [65]:
#get Status
def Status(content):
    Status = soup.findAll('span', attrs = {'class' : 'status-container', 'data-rf-test-id': 'abp-status'})
    statusList = []
    for i in Status:
        statusList.append(i.get_text())
        
    statusForHouse = []
    for j in statusList:
        statusForHouse.append(j.split(':')[1])
        
    return statusForHouse

In [66]:
##MLS Listing
def MLSListing(content):
    mlsListingList = []

    divs = soup.findAll('div', attrs = {'class': 'keyDetail font-weight-roman font-size-base'})    
    for div in divs:
        spans = div.findAll('span')
        if len(spans) > 0:        
            if "MLS" in spans[0].get_text():
                mlsListingList.append(spans[1].get_text().replace("ML","")) 
                
    return mlsListingList

In [67]:
##Description
def Description(content):
    description = soup.findAll('p', attrs={"class": "text-base"})

    house_description = ""
    for des in description:
        house_description = des.get_text()
        
    return house_description

In [68]:
##house Payment Per month
def payment_per_Month(content):
    divinfo = content.find('div', attrs = {'class': 'CalculatorSummary'})
    price_info =divinfo.find('p').get_text().split(" ")[0]
    return price_info

### Below Modules help Inserting Scrapped Data into Panda Data Frame

### 1-populate school details to the panda dataframe

In [69]:
def write_school_toDataFrame(content,realestate_df,index):
    try:
        dct_schoolDetail=schoolDetail(content)
        for key, value in dct_schoolDetail.items() :
           # print (key, value)
            if 'Elementary' in key:
                realestate_df.loc[index]['ElementarySchool'] = key
                realestate_df.loc[index]['E.SchoolRating'] = value
            elif 'Middle' in key:
                realestate_df.loc[index]['MiddleSchool'] = key
                realestate_df.loc[index]['M.schoolRating'] = value
            elif 'High' in key:
                realestate_df.loc[index]['HighSchool'] = key
                realestate_df.loc[index]['H.SchoolRating'] = value
    except Exception as  e:
        pass
                        

### 2- Populate Transit Score to DataFrame

In [70]:
# Transportation score
def write_transport_toDataFrame(content,realestate_df,index):
    try:
        dct_transitDetail=transitDetail(content)
        for key, value in dct_transitDetail.items() :
            if 'WalkScore' in key:
                realestate_df.loc[index]['WalkableScore'] = value
            elif 'TransitScore' in key:
                realestate_df.loc[index]['TransitScore'] = value
            elif 'BikeScore' in key:
                realestate_df.loc[index]['BikeableScore'] = value
    except Exception as  e:
        pass
        

### 3- Populate HOA Information to DataFrame

In [71]:
# HOA Details
def write_HOA_toDataFrame(content,realestate_df,index):
    try:
        dct_HOADetail=HOADetail(content)
        for key, value in dct_HOADetail.items() :
            if 'Hoa/Month' in key:
                realestate_df.loc[index]['Hoa/Month'] = value
    except Exception as  e:
        pass


### 4-Populate Built Year and Renovate year to DataFrame

In [72]:
# Built year and Renovated Details
def write_Built_toDataFrame(content,realestate_df,index):
    try:
        dct_BuiltDetail=BuiltDetail(content)
        for key, value in dct_BuiltDetail.items() :
            #print (key, value)
            if 'YearBuilt' in key:
                realestate_df.loc[index]['YearBuilt'] = value
            elif 'YearRenovated' in key:
                realestate_df.loc[index]['YearRenovated'] = value
    except Exception as e:
        pass
        

### 5- Populate RedFin Estimate ,Estimate ForSold Houses ,Selling Price ,House Status, Description to DataFrame


In [73]:
#redfin Estimate to dataFrame
def write_redfinEstimate_toDataFrame(content, realestate_df, index):
    try:
        redfinEstimate = redfinEstimateValue(content)

        for value in redfinEstimate:
            realestate_df.loc[index]['RedfinEstimate'] = value
    except Exception as e:
        pass
        

In [74]:
#redfin Estimate for sold houses to dataFrame
def write_redfinEstimateForSoldHouses_toDataFrame(content, realestate_df, index):
    try:
        redfinEstimateSHList = redfinEstimateForSoldHouses(content)

        for value in redfinEstimateSHList:
            realestate_df.loc[index]['RedfinEstimate'] = value
    except Exception as e:
        pass
        

In [75]:
#selling Price to dataFrame
def write_sellingPrice_toDataFrame(content, realestate_df, index):
    try:
        sellingPriceList = sellingPrice(content)

        for value in sellingPriceList:
            realestate_df.loc[index]['Price'] = value
    except Exception as e:
        pass
                       

In [76]:
#selling price of sold houses to dataFrame
def write_sellingPriceSoldHouses_toDataFrame(content, realestate_df, index):
    try:
        sellingPriceSoldHouseList = sellingPriceForSoldHouses(content)

        for value in sellingPriceSoldHouseList:
            realestate_df.loc[index]['Price'] = value
        
    except  Exception as e:
        pass
        

In [77]:
#status to dataFrame
def write_status_toDataFrame(content, realestate_df, index):
    status = "Active"
    try:
        statusForHouse = Status(content)
        if len(statusForHouse) > 0:
            status = statusForHouse[0].strip()
            realestate_df.loc[index]['Status'] = status
    except Exception as e:
        pass

    return status

In [78]:
#description to dataframe
def write_description_toDataFrame(content, realestate_df, index):
    try:
        description = Description(content)
        realestate_df.loc[index]['Description'] = description
    
    except Exception as e:
        pass
            
    return description
        


In [79]:
#paymentper month to dataframe
def write_paymentperMonth_toDataFrame(content, realestate_df, index):
    try:
        payment = payment_per_Month(content)
        realestate_df.loc[index]['payment_per_month'] = payment
    
    except Exception as e:
        pass


### 6- Populate Property Information to DataFrame


In [80]:
#Cooling Detail to DataFrame
def write_cooling_toDataFrame(content,realestate_df,index):
    try:
        Cooling_todict=Cooling_Detail(content)
        for key, value in Cooling_todict.items() :
            if 'Cooling' in key:
                realestate_df.loc[index]['Cooling'] = value
            elif 'Cooling' not in key:
                realestate_df.loc[index]['Cooling'] == 'NULL'
    except Exception as e:
        pass

#Heating Detail to DataFrame        
def write_heating_toDataFrame(content,realestate_df,index):
    try:
        Heating_todict=Heating_Detail(content)
        for key, value in Heating_todict.items() :
            if 'Heating' in key:
                realestate_df.loc[index]['Heating'] = value
            elif 'Heating' not in key:
                realestate_df.loc[index]['Heating'] == 'NULL'    
    except Exception as e:
        pass
        
#Living Sq. Ft. Detail to DataFrame        
def write_LivingSqFt_toDataFrame(content,realestate_df,index):
    try:
        LivingSqFt_todict=LivingSqFt_Detail(content)
        for key, value in LivingSqFt_todict.items() :
            if 'Living Sq. Ft.' in key:
                realestate_df.loc[index]['Living_Sq_Ft'] = value
            elif 'Living Sq. Ft.' not in key:
                realestate_df.loc[index]['Living_Sq_Ft'] == 'NULL'  
    except Exception as e:
        pass

#Lot Acres Detail to DataFrame 
def write_LotAcres_toDataFrame(content,realestate_df,index):
    try:
        LotAcres_todict=LotAcres_Detail(content)
        for key, value in LotAcres_todict.items() :
            if 'Lot Acres' in key:
                realestate_df.loc[index]['Lot_Acres'] = value
            elif 'Lot Acres' not in key:
                realestate_df.loc[index]['Lot_Acres'] == 'NULL'
    except Exception as e:
        pass


In [81]:
#Property_Type Detail to DataFrame        
# Property Type

def write_PropertyType_toDataFrame(content,realestate_df,index):
    try:
        dct_PropertyDetail=PropertyDetail(content)
        for key, value in dct_PropertyDetail.items() :
            if 'PropertyType' in key:
                realestate_df.loc[index]['PropertyType'] = value
    except Exception as e:
        pass


In [82]:
def write_noOfBeds_toDataFrame(content,realestate_df,index):
    try:
        beds_List = bedsDetail(content)
        for value in beds_List:
            realestate_df.loc[index]['Beds'] = value
    except Exception as e:
        pass
        
def write_noOfBaths_toDataFrame(content,realestate_df,index):
    try:
        baths_List = bathsDetail(content) 
        for value in baths_List:
            realestate_df.loc[index]['Baths'] = value
    except Exception as e:
        pass
        
def write_parking_toDataFrame(content,realestate_df,index):
    try:
        parking_todict=parkingDetail(content)
        for key, value in parking_todict.items() :
            if 'Parking' in key:
                realestate_df.loc[index]['Parking'] = value
            elif 'Parking' not in key:
                realestate_df.loc[index]['Parking'] == 'NULL'
    except Exception as e:
        pass

def write_garage_toDataFrame(content,realestate_df,index):
    try:
        garage_todict=garageDetail(content)
        for key, value in garage_todict.items() :
            if 'Garage' in key:
                realestate_df.loc[index]['Garage'] = value
            elif 'Garage' not in key:
                realestate_df.loc[index]['Garage'] == 'NULL'
    except Exception as e:
        pass


In [83]:
def write_MLSListing_toDataFrame(content, realestate_df, index):
    try:
        mlsListingList = MLSListing(content)
    
        for MLS in mlsListingList:
            realestate_df.loc[index]['MLS Listing'] = MLS
    except Exception as e:
        pass


## Main function start here
### Includes DataFrame Creation 
### In the Following Section Html files for source directory are scrapped using Beautiful Soup
### the source file Direcory is available in the following Link
### https://drive.google.com/drive/folders/1msDCvSEulqba9ysS40ex0kkD7mEIBTNE?usp=sharing
### Please download the html files directory  to  local system and set the path before proceeding with following code

In [84]:

# main function starts here
# Reading html file from the directory using BeautifulSoup

column_name =['PropertyType','Address','City','State','ZipCode','Price','Beds','Baths',
             'Living_Sq_Ft','Lot_Acres','YearBuilt','YearRenovated','Hoa/Month','Parking','Garage',
             'ElementarySchool','E.SchoolRating','MiddleSchool',
             'M.schoolRating','HighSchool','H.SchoolRating','Heating','Cooling','WalkableScore',
             'TransitScore','BikeableScore','RedfinEstimate', 'Status', 'MLS Listing', 'Description','payment_per_month']
realestate_df = pd.DataFrame(columns =column_name)
counter=0
os.chdir('/Users/arunchockalingam/Downloads/DA_FinalProject_data')
fileList = os.listdir()
numberOfFiles = len(fileList)

for file in fileList:
    try:
        soup = BeautifulSoup(open(file,encoding='utf-8'), "html.parser")
        house_index ="house_"+str(counter)
        realestate_df= realestate_df.append(pd.Series(name=house_index))
        pd.options.mode.chained_assignment = None

        write_noOfBeds_toDataFrame(soup,realestate_df,house_index)

        write_noOfBaths_toDataFrame(soup,realestate_df,house_index)

        write_parking_toDataFrame(soup,realestate_df,house_index)

        write_garage_toDataFrame(soup,realestate_df,house_index)

        write_school_toDataFrame(soup,realestate_df,house_index)

        write_transport_toDataFrame(soup,realestate_df,house_index)

        write_HOA_toDataFrame(soup,realestate_df,house_index)

        write_Built_toDataFrame(soup,realestate_df,house_index)

        write_cooling_toDataFrame(soup,realestate_df,house_index)

        write_heating_toDataFrame(soup,realestate_df,house_index)

        write_LivingSqFt_toDataFrame(soup,realestate_df,house_index)

        write_LotAcres_toDataFrame(soup,realestate_df,house_index)

        write_PropertyType_toDataFrame(soup,realestate_df,house_index)

        get_house_details(soup,realestate_df,house_index)

        status = write_status_toDataFrame(soup, realestate_df, house_index)
        if(status == 'Sold'):            
            write_redfinEstimateForSoldHouses_toDataFrame(soup, realestate_df, house_index)
            write_sellingPriceSoldHouses_toDataFrame(soup, realestate_df, house_index)

        else:
            write_redfinEstimate_toDataFrame(soup, realestate_df, house_index)
            write_sellingPrice_toDataFrame(soup, realestate_df, house_index)
            
        write_MLSListing_toDataFrame(soup, realestate_df, house_index)
        
        write_description_toDataFrame(soup, realestate_df, house_index)
        
        write_paymentperMonth_toDataFrame(soup, realestate_df, house_index)
    
    except Exception as e:
        print("error in File :",file)
        print(e)
    
    except:
        print(file)
    
    finally:
        counter=counter+1
        print("Processing Property: ",counter,"/",numberOfFiles)


Processing Property:  1 / 1697
Processing Property:  2 / 1697
Processing Property:  3 / 1697
Processing Property:  4 / 1697
Processing Property:  5 / 1697
Processing Property:  6 / 1697
Processing Property:  7 / 1697
Processing Property:  8 / 1697
Processing Property:  9 / 1697
Processing Property:  10 / 1697
Processing Property:  11 / 1697
Processing Property:  12 / 1697
Processing Property:  13 / 1697
Processing Property:  14 / 1697
Processing Property:  15 / 1697
Processing Property:  16 / 1697
Processing Property:  17 / 1697
Processing Property:  18 / 1697
Processing Property:  19 / 1697
Processing Property:  20 / 1697
Processing Property:  21 / 1697
Processing Property:  22 / 1697
Processing Property:  23 / 1697
Processing Property:  24 / 1697
Processing Property:  25 / 1697
Processing Property:  26 / 1697
Processing Property:  27 / 1697
Processing Property:  28 / 1697
Processing Property:  29 / 1697
Processing Property:  30 / 1697
Processing Property:  31 / 1697
Processing Proper

Processing Property:  253 / 1697
Processing Property:  254 / 1697
Processing Property:  255 / 1697
Processing Property:  256 / 1697
Processing Property:  257 / 1697
Processing Property:  258 / 1697
Processing Property:  259 / 1697
Processing Property:  260 / 1697
Processing Property:  261 / 1697
Processing Property:  262 / 1697
Processing Property:  263 / 1697
Processing Property:  264 / 1697
Processing Property:  265 / 1697
Processing Property:  266 / 1697
Processing Property:  267 / 1697
Processing Property:  268 / 1697
Processing Property:  269 / 1697
Processing Property:  270 / 1697
Processing Property:  271 / 1697
Processing Property:  272 / 1697
Processing Property:  273 / 1697
Processing Property:  274 / 1697
Processing Property:  275 / 1697
Processing Property:  276 / 1697
Processing Property:  277 / 1697
Processing Property:  278 / 1697
Processing Property:  279 / 1697
Processing Property:  280 / 1697
Processing Property:  281 / 1697
Processing Property:  282 / 1697
Processing

Processing Property:  502 / 1697
Processing Property:  503 / 1697
Processing Property:  504 / 1697
Processing Property:  505 / 1697
Processing Property:  506 / 1697
Processing Property:  507 / 1697
Processing Property:  508 / 1697
Processing Property:  509 / 1697
Processing Property:  510 / 1697
Processing Property:  511 / 1697
Processing Property:  512 / 1697
Processing Property:  513 / 1697
Processing Property:  514 / 1697
Processing Property:  515 / 1697
Processing Property:  516 / 1697
Processing Property:  517 / 1697
Processing Property:  518 / 1697
Processing Property:  519 / 1697
Processing Property:  520 / 1697
Processing Property:  521 / 1697
Processing Property:  522 / 1697
Processing Property:  523 / 1697
Processing Property:  524 / 1697
Processing Property:  525 / 1697
Processing Property:  526 / 1697
Processing Property:  527 / 1697
Processing Property:  528 / 1697
Processing Property:  529 / 1697
Processing Property:  530 / 1697
Processing Property:  531 / 1697
Processing

Processing Property:  751 / 1697
Processing Property:  752 / 1697
Processing Property:  753 / 1697
Processing Property:  754 / 1697
Processing Property:  755 / 1697
Processing Property:  756 / 1697
Processing Property:  757 / 1697
Processing Property:  758 / 1697
Processing Property:  759 / 1697
Processing Property:  760 / 1697
Processing Property:  761 / 1697
Processing Property:  762 / 1697
Processing Property:  763 / 1697
Processing Property:  764 / 1697
Processing Property:  765 / 1697
Processing Property:  766 / 1697
Processing Property:  767 / 1697
Processing Property:  768 / 1697
Processing Property:  769 / 1697
Processing Property:  770 / 1697
Processing Property:  771 / 1697
Processing Property:  772 / 1697
Processing Property:  773 / 1697
Processing Property:  774 / 1697
Processing Property:  775 / 1697
Processing Property:  776 / 1697
Processing Property:  777 / 1697
Processing Property:  778 / 1697
Processing Property:  779 / 1697
Processing Property:  780 / 1697
Processing

Processing Property:  1000 / 1697
Processing Property:  1001 / 1697
Processing Property:  1002 / 1697
Processing Property:  1003 / 1697
Processing Property:  1004 / 1697
Processing Property:  1005 / 1697
Processing Property:  1006 / 1697
Processing Property:  1007 / 1697
Processing Property:  1008 / 1697
Processing Property:  1009 / 1697
Processing Property:  1010 / 1697
Processing Property:  1011 / 1697
Processing Property:  1012 / 1697
Processing Property:  1013 / 1697
Processing Property:  1014 / 1697
Processing Property:  1015 / 1697
Processing Property:  1016 / 1697
Processing Property:  1017 / 1697
Processing Property:  1018 / 1697
Processing Property:  1019 / 1697
Processing Property:  1020 / 1697
Processing Property:  1021 / 1697
Processing Property:  1022 / 1697
Processing Property:  1023 / 1697
Processing Property:  1024 / 1697
Processing Property:  1025 / 1697
Processing Property:  1026 / 1697
Processing Property:  1027 / 1697
Processing Property:  1028 / 1697
Processing Pro

Processing Property:  1241 / 1697
Processing Property:  1242 / 1697
Processing Property:  1243 / 1697
Processing Property:  1244 / 1697
Processing Property:  1245 / 1697
Processing Property:  1246 / 1697
Processing Property:  1247 / 1697
Processing Property:  1248 / 1697
Processing Property:  1249 / 1697
Processing Property:  1250 / 1697
Processing Property:  1251 / 1697
Processing Property:  1252 / 1697
Processing Property:  1253 / 1697
Processing Property:  1254 / 1697
Processing Property:  1255 / 1697
Processing Property:  1256 / 1697
Processing Property:  1257 / 1697
Processing Property:  1258 / 1697
Processing Property:  1259 / 1697
Processing Property:  1260 / 1697
Processing Property:  1261 / 1697
Processing Property:  1262 / 1697
Processing Property:  1263 / 1697
Processing Property:  1264 / 1697
Processing Property:  1265 / 1697
Processing Property:  1266 / 1697
Processing Property:  1267 / 1697
Processing Property:  1268 / 1697
Processing Property:  1269 / 1697
Processing Pro

Processing Property:  1482 / 1697
Processing Property:  1483 / 1697
Processing Property:  1484 / 1697
Processing Property:  1485 / 1697
Processing Property:  1486 / 1697
Processing Property:  1487 / 1697
Processing Property:  1488 / 1697
Processing Property:  1489 / 1697
Processing Property:  1490 / 1697
Processing Property:  1491 / 1697
Processing Property:  1492 / 1697
Processing Property:  1493 / 1697
Processing Property:  1494 / 1697
Processing Property:  1495 / 1697
Processing Property:  1496 / 1697
Processing Property:  1497 / 1697
Processing Property:  1498 / 1697
Processing Property:  1499 / 1697
Processing Property:  1500 / 1697
Processing Property:  1501 / 1697
Processing Property:  1502 / 1697
Processing Property:  1503 / 1697
Processing Property:  1504 / 1697
Processing Property:  1505 / 1697
Processing Property:  1506 / 1697
Processing Property:  1507 / 1697
Processing Property:  1508 / 1697
Processing Property:  1509 / 1697
Processing Property:  1510 / 1697
Processing Pro

###  Check 5 rows of DataFrame

In [90]:
realestate_df.head()

,PropertyType,Address,City,State,ZipCode,Price,Beds,Baths,Living_Sq_Ft,Lot_Acres,...,Heating,Cooling,WalkableScore,TransitScore,BikeableScore,RedfinEstimate,Status,MLS Listing,Description,payment_per_month
house_0,Condo/Co-op,35 Dolores St #103,San Francisco,CA,94103,312324,2,2,—,NaN,...,NaN,NaN,98,100,90,"$322,211",Active,494553,2 bedroom Below Market Rate (BMR) housing oppo...,"$2,083"
house_1,Single Family Residential,34 George St,SAN JOSE,CA,95110,1349900,4,3,"2,544",0.1064,...,Central Forced Air,Central Forced Air,84,65,88,"$1,351,241",Active,81782737,Beautifully updated home in desirable San Jose...,"$6,731"
house_2,Single Family Residential,27 Saturn St,San Francisco,CA,94114,1995000,3,2,1950,NaN,...,NaN,NaN,90,89,64,"$2,366,967",Active,495259,Stunning 3BR/2BA Mid-Century Modern Corona Hei...,"$10,012"
house_3,Single Family Residential,3909 Avocet Ter,Fremont,CA,94555,840000,2,2,1407,NaN,...,NaN,NaN,54,31,59,"$883,191",Active,40895548,A great home featuring a double door entryway ...,"$4,440"
house_4,1954,1223 Nilda Ave,MOUNTAIN VIEW,CA,94040,0,3,2,"1,414",0.1393,...,Central Forced Air,Central Forced Air,81,43,95,2231863,Sold,81768941,Located in a highly desired street in Cuesta P...,"$10,507"


### Code Cleaning

In [86]:
# HOA Cleaning 
def HoaToInt(string):
    try:
        #print(string)
        length=len(string)
        if string=='NULL' :
            return 0 
        else: 
            return int(string.split('$')[1].split('/')[0].replace(',',''))
    except:
        return 0

realestate_df['Hoa/Month']=realestate_df['Hoa/Month'].apply(HoaToInt)

# Price Cleaning
def PriceToInt(string):
    try:
        string=str(string)
        return int(string.split('$')[1].split('+')[0].replace(',',''))
    except:
        return 0

realestate_df['Price']=realestate_df['Price'].apply(PriceToInt)

In [87]:
#remove rows with null values address columns
def removeRowswithNullAddressValues(realestate_df):
    realestate_df = realestate_df[realestate_df['Address'].notnull()]
    realestate_df['ZipCode'].replace('', np.nan, inplace=True)
    realestate_df = realestate_df[realestate_df['ZipCode'].notnull()]
    return realestate_df

realestate_df = removeRowswithNullAddressValues(realestate_df)

In [88]:
# # ZipCode Cleaning
def RemoveZipExt(string):
    try:
        string=str(string)
        return string.split('-')[0]
    except:
        return string

realestate_df['ZipCode']=realestate_df['ZipCode'].apply(RemoveZipExt)

### DataFrame is saved as Csv File for Analysis and Design

In [91]:
realestate_df.to_csv('realestate.csv')